<a href="https://colab.research.google.com/github/nhuyen183/LungCancerSupportSystem/blob/master/annFinalipynb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
### start spark session

In [ ]:
####Once Everything Is Installed

In [ ]:
!pip install pyspark

     |████████████████████████████████| 212.3MB 68kB/s 
     |████████████████████████████████| 204kB 19.9MB/s 
  Created wheel for pyspark: filename=pyspark-3.1.1-py2.py3-none-any.whl size=212767604 sha256=abc4b566b87ccc7fa87ba29427a05e76f40a8da89cd8088a809480371f22fd86
  Stored in directory: /root/.cache/pip/wheels/0b/90/c0/01de724414ef122bd05f056541fb6a0ecf47c7ca655f8b3c0f
Successfully built pyspark


In [ ]:
!pip install pyarrow ## pandas dataframe

In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.master('local').getOrCreate() ##spark object

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
path='/content/drive/MyDrive/1_GoogleCoLab/Data_Code/section6/dl_data.csv'

In [ ]:
df1=spark.read.csv(path,inferSchema=True, header=True)

In [ ]:
df1.show(5)

+-------------------+---------------------+-----------------+-------------------------------------+-------------------------------+-----------------------+------------------------------+-------------------------------+-------------------------+-------------------------+-------------------+------------------+-------------------+-----------------+------------------+------------------+-------------------------+
|Visit_Number_Bucket|Page_Views_Normalized|Orders_Normalized|Internal_Search_Successful_Normalized|Internal_Search_Null_Normalized|Email_Signup_Normalized|Total_Seconds_Spent_Normalized|Store_Locator_Search_Normalized|Mapped_Last_Touch_Channel|Mapped_Mobile_Device_Type|Mapped_Browser_Type|Mapped_Entry_Pages|Mapped_Site_Section|Mapped_Promo_Code|Maped_Product_Name|Mapped_Search_Term|Mapped_Product_Collection|
+-------------------+---------------------+-----------------+-------------------------------------+-------------------------------+-----------------------+---------------------

In [ ]:
data = df1.withColumnRenamed('Orders_Normalized', 'label') #rename columns

In [ ]:
from pyspark.ml.feature import OneHotEncoder, VectorAssembler, StringIndexer
from pyspark.ml import Pipeline
from pyspark.sql.functions import udf, StringType
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml.classification import MultilayerPerceptronClassifier

In [ ]:
train, validation, test = data.randomSplit([0.7, 0.2, 0.1], 1234)

In [ ]:
categorical_columns = [item[0] for item in data.dtypes if item[1].startswith(
    'string')]
numeric_columns = [item[0] for item in data.dtypes if item[1].startswith(
    'double')]
indexers = [StringIndexer(inputCol=column, outputCol='{0}_index'.format(
    column)) for column in categorical_columns] ## read in the string columns

In [ ]:
featuresCreator = VectorAssembler(
    inputCols=[indexer.getOutputCol() for indexer in indexers] + numeric_columns,
    outputCol='features')
layers = [len(featuresCreator.getInputCols()), 4, 2, 2] ## no. of predictors, hidden nodes, response

In [ ]:
classifier = MultilayerPerceptronClassifier(labelCol='label',
                                            featuresCol='features',
                                            maxIter=100,
                                            layers=layers,
                                            blockSize=128,
                                            seed=1234)

In [ ]:
pipeline = Pipeline(stages=indexers + [featuresCreator, classifier])

In [ ]:
model = pipeline.fit(train)

In [ ]:
train_output_df = model.transform(train)
validation_output_df = model.transform(validation)
test_output_df = model.transform(test)

In [ ]:
train_predictionAndLabels = train_output_df.select('prediction', 'label')
validation_predictionAndLabels = validation_output_df.select('prediction', 'label')
test_predictionAndLabels = test_output_df.select('prediction', 'label')


In [ ]:
metrics = ['weightedPrecision', 'weightedRecall', 'accuracy']
for metric in metrics:
  evaluator = MulticlassClassificationEvaluator(metricName=metric)
  print('Train ' + metric + ' = ' + str(evaluator.evaluate(
      train_predictionAndLabels)))
  print('Validation ' + metric + ' = ' + str(evaluator.evaluate(
      validation_predictionAndLabels)))
  print('Test ' + metric + ' = ' + str(evaluator.evaluate(
      test_predictionAndLabels)))

Train weightedPrecision = 0.9719688384869849
Validation weightedPrecision = 0.9707293561470658
Test weightedPrecision = 0.9706661982088725
Train weightedRecall = 0.9708813302511704
Validation weightedRecall = 0.9694943015082236
Test weightedRecall = 0.9694085656016316
Train accuracy = 0.9708813302511704
Validation accuracy = 0.9694943015082236
Test accuracy = 0.9694085656016316
